In [1]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
from evaluate import evaluator
from transformers import pipeline
import torch

2023-10-12 10:18:35.334039: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 10:18:35.373394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 10:18:35.794145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = "cuda"
net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to(device)
def mbt2018_compress(sample):
    return nn_compress(sample,net_mbt2018,device)

In [25]:
image_compression_methods = [
    jpeg_compress,
    webp_compress,
    mbt2018_compress,
    hific_lo_compress
]
models = [
    "microsoft/resnet-18",
    "google/vit-base-patch16-224",
    "microsoft/beit-large-patch16-224"
]

In [4]:
imagenet = [load_dataset("imagenet-1k", split="validation[:1000]")]

In [5]:
for method in image_compression_methods:
    imagenet.append(imagenet[0].map(method))

In [45]:
pipe = []
for model in models:
    pipe.append(
        pipeline(
            task="image-classification",
            model=model,
            device="cuda:0"
        )
    )

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [46]:
task_evaluator = evaluator("image-classification")

In [47]:
eval_results = []
for i_method,method in enumerate(imagenet):
    eval_results.append([])
    for i_model,model in enumerate(pipe):
        eval_results[i_method].append(
            task_evaluator.compute(
                model_or_pipeline=model,
                data=method,
                metric="accuracy",
                label_mapping=model.model.config.label2id
            )
        )

In [48]:
[[rij['accuracy'] for rij in r] for r in eval_results]

[[0.698, 0.799, 0.873],
 [0.394, 0.639, 0.794],
 [0.505, 0.72, 0.819],
 [0.488, 0.664, 0.756],
 [0.688, 0.796, 0.873]]